**Vehicle Classification**

**Evaluate your model**

In [ ]:
import os

from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50

**Experiment settings**

Set here the two variables in the following way:

CONFIG_YML: assign the path to the config.yml file used for the experiment 

1.   CONFIG_YML: assign the path to the config.yml file used for the experiment you want to evaluate

2.   WEIGHTS: assign the path to the model weights (.h5 file) you want to evaluate

In [ ]:
# Modify this
CONFIG_YML = "../experiments/exp_019/config_exp019.yml"
WEIGHTS = "../experiments/exp_019/model.57-0.7264.h5"

In [ ]:
# (!) Don't touch this cell, will load the config file specified before
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v2/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 50},
 'model': {'weights': '../experiments/exp_019/model.57-0.7264.h5',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.35,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.075},
   'random_zoom': {'height_factor': 0.15, 'width_factor': 0.15}}},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.0001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 100,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_019/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': False},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp_019/logs'}}}}

In [ ]:
# (!) Don't touch this cell, will use the config file to infer the class names
#     and also to load the corresponding testing dataset.
#     If you get an error, you must review your data/code.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

**Load your model**

Use resnet_50.create_model() and remember to properly setup the model weights!

Assign the model to the variable cnn_model.

In [ ]:
# Complete this

cnn_model = resnet_50.create_model(**config["model"])

# It should print your model correctly
print(cnn_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 196)               401604    
                                                                 
=================================================================
Total params: 23,989,316
Trainable params: 23,936,196
Non-trainable params: 53,120
_________________________________________________________________
None

**Get predictions from testing dataset**

In [ ]:
# (!) Don't touch this cell, will use the loaded model and
#     the function utils.predict_from_folder() to get 
#     model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

In [ ]:
# (!) Don't touch this cell

print(classification_report(y_true=labels, y_pred=predictions))

precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.68      0.82      0.74        44
                             Acura Integra Type R 2001       0.68      0.48      0.56        44
                                   Acura RL Sedan 2012       0.54      0.22      0.31        32
                                   Acura TL Sedan 2012       0.85      0.67      0.75        43
                                  Acura TL Type-S 2008       0.74      0.55      0.63        42
                                  Acura TSX Sedan 2012       0.84      0.53      0.65        40
                              Acura ZDX Hatchback 2012       0.45      0.62      0.52        39
              Aston Martin V8 Vantage Convertible 2012       0.28      0.42      0.34        45
                    Aston Martin V8 Vantage Coupe 2012       0.31      0.39      0.35        41
                  Aston Martin Virage Convertible 2012       0.50      0.61      0.55        33
                        Aston Martin Virage Coupe 2012       0.40      0.84      0.54        38
                                   Audi 100 Sedan 1994       0.44      0.30      0.36        40
                                   Audi 100 Wagon 1994       0.13      0.26      0.18        42
                                    Audi A5 Coupe 2012       0.51      0.49      0.50        41
                                    Audi R8 Coupe 2012       0.87      0.47      0.61        43
                            Audi RS 4 Convertible 2008       0.36      0.69      0.48        36
                                    Audi S4 Sedan 2007       0.67      0.36      0.46        45
                                    Audi S4 Sedan 2012       0.34      0.38      0.36        39
                              Audi S5 Convertible 2012       0.80      0.19      0.31        42
                                    Audi S5 Coupe 2012       0.34      0.24      0.28        42
                                    Audi S6 Sedan 2011       0.46      0.70      0.55        46
                                Audi TT Hatchback 2011       0.35      0.42      0.39        40
                                 Audi TT RS Coupe 2012       0.51      0.49      0.50        39
                                   Audi TTS Coupe 2012       0.20      0.36      0.25        42
                                    Audi V8 Sedan 1994       0.19      0.49      0.27        43
                         BMW 1 Series Convertible 2012       0.76      0.54      0.63        35
                               BMW 1 Series Coupe 2012       0.40      0.78      0.53        41
                               BMW 3 Series Sedan 2012       0.63      0.64      0.64        42
                               BMW 3 Series Wagon 2012       0.52      0.54      0.53        41
                         BMW 6 Series Convertible 2007       0.60      0.27      0.37        44
                         BMW ActiveHybrid 5 Sedan 2012       1.00      0.47      0.64        34
                                     BMW M3 Coupe 2012       0.90      0.59      0.71        44
                                     BMW M5 Sedan 2010       0.90      0.46      0.61        41
                               BMW M6 Convertible 2010       0.37      0.63      0.47        41
                                       BMW X3 SUV 2012       0.69      0.71      0.70        38
                                       BMW X5 SUV 2007       0.74      0.63      0.68        41
                                       BMW X6 SUV 2012       0.59      0.55      0.57        42
                               BMW Z4 Convertible 2012       0.64      0.23      0.33        40
                             Bentley Arnage Sedan 2009       0.73      0.56      0.64        39
            Bentley Continental Flying Spur Sedan 2007       0.42      0.50      0.46        44
                     Bentley Continental GT Coupe 2007       0.50      0.46      0.48        46
                     Bentley Continental GT Coupe 2012       0.78      0.41      0.54        34
Bentley Continental Supersports Conv. Convertible 2012       0.76      0.44      0.56        36
                           Bentley Mulsanne Sedan 2011       0.84      0.60      0.70        35
                  Bugatti Veyron 16.4 Convertible 2009       0.68      0.59      0.63        32
                        Bugatti Veyron 16.4 Coupe 2009       0.67      0.51      0.58        43
                                Buick Enclave SUV 2012       0.71      0.69      0.70        42
                                Buick Rainier SUV 2007       0.95      0.50      0.66        42
                                   Buick Regal GS 2012       0.63      0.49      0.55        35
                               Buick Verano Sedan 2012       0.61      0.38      0.47        37
                             Cadillac CTS-V Sedan 2012       0.88      0.53      0.67        43
                   Cadillac Escalade EXT Crew Cab 2007       0.81      0.57      0.67        44
                                 Cadillac SRX SUV 2012       0.96      0.56      0.71        41
                     Chevrolet Avalanche Crew Cab 2012       0.44      0.58      0.50        45
                     Chevrolet Camaro Convertible 2012       0.76      0.50      0.60        44
                              Chevrolet Cobalt SS 2010       0.59      0.41      0.49        41
                   Chevrolet Corvette Convertible 2012       0.48      0.41      0.44        39
       Chevrolet Corvette Ron Fellows Edition Z06 2007       0.66      0.73      0.69        37
                           Chevrolet Corvette ZR1 2012       0.57      0.59      0.58        46
                      Chevrolet Express Cargo Van 2007       0.13      0.90      0.23        29
                            Chevrolet Express Van 2007       0.18      0.06      0.09        35
                                 Chevrolet HHR SS 2010       0.58      0.69      0.63        36
                           Chevrolet Impala Sedan 2007       0.89      0.19      0.31        43
                    Chevrolet Malibu Hybrid Sedan 2010       0.56      0.26      0.36        38
                           Chevrolet Malibu Sedan 2007       0.95      0.45      0.62        44
                      Chevrolet Monte Carlo Coupe 2007       0.72      0.47      0.57        45
    Chevrolet Silverado 1500 Classic Extended Cab 2007       0.93      0.62      0.74        42
            Chevrolet Silverado 1500 Extended Cab 2012       0.20      0.05      0.08        43
         Chevrolet Silverado 1500 Hybrid Crew Cab 2012       0.48      0.30      0.37        40
             Chevrolet Silverado 1500 Regular Cab 2012       0.38      0.55      0.44        44
           Chevrolet Silverado 2500HD Regular Cab 2012       0.67      0.16      0.26        38
                            Chevrolet Sonic Sedan 2012       0.35      0.86      0.49        44
                       Chevrolet Tahoe Hybrid SUV 2012       0.83      0.14      0.23        37
                         Chevrolet TrailBlazer SS 2009       0.70      0.57      0.63        40
                           Chevrolet Traverse SUV 2012       0.57      0.55      0.56        44
                               Chrysler 300 SRT-8 2010       0.90      0.40      0.55        48
                               Chrysler Aspen SUV 2009       0.81      0.58      0.68        43
                   Chrysler Crossfire Convertible 2008       0.65      0.56      0.60        43
                  Chrysler PT Cruiser Convertible 2008       0.51      0.69      0.58        45
                     Chrysler Sebring Convertible 2010       0.94      0.40      0.56        40
                Chrysler Town and Country Minivan 2012       0.90      0.49      0.63        37
                              Daewoo Nubira Wagon 2002       0.79      0.49      0.60        45
                              Dodge Caliber Wagon 2007       0.54      0.74      0.63        42
                              Dodge Caliber Wagon 2012       0.56      0.35      0.43        40
                            Dodge Caravan Minivan 1997       0.97      0.74      0.84        43
                            Dodge Challenger SRT8 2011       0.43      0.90      0.58        39
                              Dodge Charger SRT-8 2009       0.56      0.36      0.43        42
                              Dodge Charger Sedan 2012       0.68      0.56      0.61        41
                            Dodge Dakota Club Cab 2007       0.64      0.47      0.55        38
                            Dodge Dakota Crew Cab 2010       0.86      0.29      0.44        41
                                Dodge Durango SUV 2007       0.90      0.60      0.72        45
                                Dodge Durango SUV 2012       0.31      0.88      0.46        43
                                Dodge Journey SUV 2012       0.57      0.89      0.69        44
                               Dodge Magnum Wagon 2008       0.43      0.55      0.48        40
                   Dodge Ram Pickup 3500 Crew Cab 2010       0.83      0.69      0.75        42
                   Dodge Ram Pickup 3500 Quad Cab 2009       0.87      0.30      0.44        44
                         Dodge Sprinter Cargo Van 2009       0.45      0.49      0.47        39
                            Eagle Talon Hatchback 1998       0.43      0.57      0.49        46
                                  FIAT 500 Abarth 2012       0.56      0.81      0.67        27
                             FIAT 500 Convertible 2012       0.79      0.82      0.81        33
                   Ferrari 458 Italia Convertible 2012       0.69      0.23      0.35        39
                         Ferrari 458 Italia Coupe 2012       0.53      0.40      0.46        42
                   Ferrari California Convertible 2012       0.83      0.62      0.71        39
                                 Ferrari FF Coupe 2012       0.74      0.60      0.66        42
                               Fisker Karma Sedan 2012       0.95      0.47      0.62        43
                          Ford E-Series Wagon Van 2012       0.90      0.95      0.92        37
                                    Ford Edge SUV 2012       0.67      0.60      0.63        43
                           Ford Expedition EL SUV 2009       1.00      0.59      0.74        44
                           Ford F-150 Regular Cab 2007       0.96      0.60      0.74        45
                           Ford F-150 Regular Cab 2012       0.84      0.86      0.85        42
                   Ford F-450 Super Duty Crew Cab 2012       0.72      0.63      0.68        41
                                Ford Fiesta Sedan 2012       1.00      0.60      0.75        42
                                 Ford Focus Sedan 2007       1.00      0.60      0.75        45
                            Ford Freestar Minivan 2007       0.95      0.82      0.88        44
                                    Ford GT Coupe 2006       0.53      0.40      0.46        45
                         Ford Mustang Convertible 2007       0.73      0.43      0.54        44
                             Ford Ranger SuperCab 2011       0.87      0.64      0.74        42
                                   GMC Acadia SUV 2012       0.67      0.82      0.73        44
                          GMC Canyon Extended Cab 2012       0.94      0.40      0.56        40
                                   GMC Savana Van 2012       0.31      0.62      0.41        68
                                  GMC Terrain SUV 2012       0.57      0.73      0.64        41
                             GMC Yukon Hybrid SUV 2012       0.82      0.55      0.66        42
                            Geo Metro Convertible 1993       0.57      0.68      0.62        44
                           HUMMER H2 SUT Crew Cab 2009       0.40      0.63      0.49        43
                              HUMMER H3T Crew Cab 2010       0.83      0.49      0.61        39
                               Honda Accord Coupe 2012       0.56      0.46      0.51        39
                               Honda Accord Sedan 2012       0.86      0.16      0.27        38
                            Honda Odyssey Minivan 2007       0.78      0.51      0.62        41
                            Honda Odyssey Minivan 2012       0.76      0.60      0.67        42
                             Hyundai Accent Sedan 2012       0.32      0.29      0.30        24
                              Hyundai Azera Sedan 2012       0.61      0.60      0.60        42
                            Hyundai Elantra Sedan 2007       0.90      0.62      0.73        42
                Hyundai Elantra Touring Hatchback 2012       0.85      0.55      0.67        42
                            Hyundai Genesis Sedan 2012       0.88      0.65      0.75        43
                             Hyundai Santa Fe SUV 2012       0.96      0.55      0.70        42
                      Hyundai Sonata Hybrid Sedan 2012       0.88      0.85      0.86        33
                             Hyundai Sonata Sedan 2012       0.95      0.46      0.62        39
                               Hyundai Tucson SUV 2012       0.74      0.67      0.71        43
                       Hyundai Veloster Hatchback 2012       0.61      0.46      0.53        41
                             Hyundai Veracruz SUV 2012       0.46      0.57      0.51        42
                             Infiniti G Coupe IPL 2012       0.90      0.26      0.41        34
                                Infiniti QX56 SUV 2011       0.62      0.62      0.62        32
                               Isuzu Ascender SUV 2008       0.83      0.62      0.71        40
                                    Jaguar XK XKR 2012       0.91      0.43      0.59        46
                                 Jeep Compass SUV 2012       0.36      0.88      0.51        42
                          Jeep Grand Cherokee SUV 2012       0.30      0.67      0.42        45
                                 Jeep Liberty SUV 2012       0.75      0.55      0.63        44
                                 Jeep Patriot SUV 2012       0.63      0.82      0.71        44
                                Jeep Wrangler SUV 2012       0.62      0.95      0.75        43
                      Lamborghini Aventador Coupe 2012       0.78      0.33      0.46        43
                         Lamborghini Diablo Coupe 2001       0.74      0.64      0.68        44
       Lamborghini Gallardo LP 570-4 Superleggera 2012       0.82      0.66      0.73        35
                       Lamborghini Reventon Coupe 2008       0.47      0.75      0.57        36
                               Land Rover LR2 SUV 2012       0.82      0.74      0.78        42
                       Land Rover Range Rover SUV 2012       0.38      0.93      0.53        42
                           Lincoln Town Car Sedan 2011       0.56      0.56      0.56        39
                 MINI Cooper Roadster Convertible 2012       0.74      0.81      0.77        36
                    Maybach Landaulet Convertible 2012       0.51      0.69      0.59        29
                                Mazda Tribute SUV 2011       0.59      0.56      0.57        36
                            McLaren MP4-12C Coupe 2012       0.93      0.59      0.72        44
              Mercedes-Benz 300-Class Convertible 1993       0.52      0.73      0.61        48
                      Mercedes-Benz C-Class Sedan 2012       0.89      0.53      0.67        45
                      Mercedes-Benz E-Class Sedan 2012       0.49      0.63      0.55        43
                      Mercedes-Benz S-Class Sedan 2012       0.89      0.39      0.54        44
                     Mercedes-Benz SL-Class Coupe 2009       1.00      0.42      0.59        36
                       Mercedes-Benz Sprinter Van 2012       0.48      0.88      0.62        41
                          Mitsubishi Lancer Sedan 2012       0.63      0.70      0.67        47
                               Nissan 240SX Coupe 1998       0.67      0.70      0.68        46
                            Nissan Juke Hatchback 2012       0.26      0.93      0.41        44
                            Nissan Leaf Hatchback 2012       0.70      0.79      0.74        42
                          Nissan NV Passenger Van 2012       0.61      0.79      0.69        38
                              Plymouth Neon Coupe 1999       0.61      0.80      0.69        44
                           Porsche Panamera Sedan 2012       0.69      0.42      0.52        43
                        Ram C-V Cargo Van Minivan 2012       0.52      0.73      0.61        41
                          Rolls-Royce Ghost Sedan 2012       0.66      0.55      0.60        38
   Rolls-Royce Phantom Drophead Coupe Convertible 2012       0.60      0.50      0.55        30
                        Rolls-Royce Phantom Sedan 2012       0.63      0.55      0.59        44
                               Scion xD Hatchback 2012       0.88      0.54      0.67        41
                            Spyker C8 Convertible 2009       0.74      0.56      0.63        45
                                  Spyker C8 Coupe 2009       0.49      0.55      0.52        42
                               Suzuki Aerio Sedan 2007       0.93      0.37      0.53        38
                             Suzuki Kizashi Sedan 2012       0.56      0.65      0.60        46
                             Suzuki SX4 Hatchback 2012       0.35      0.86      0.50        42
                                 Suzuki SX4 Sedan 2012       0.72      0.33      0.45        40
                              Tesla Model S Sedan 2012       0.57      0.61      0.59        38
                               Toyota 4Runner SUV 2012       0.91      0.50      0.65        40
                               Toyota Camry Sedan 2012       0.59      0.47      0.52        43
                             Toyota Corolla Sedan 2012       0.51      0.51      0.51        43
                               Toyota Sequoia SUV 2012       0.96      0.63      0.76        38
                      Volkswagen Beetle Hatchback 2012       0.47      0.86      0.61        42
                        Volkswagen Golf Hatchback 1991       0.35      0.96      0.51        46
                        Volkswagen Golf Hatchback 2012       0.33      0.67      0.44        43
                                  Volvo 240 Sedan 1993       1.00      0.56      0.71        45
                              Volvo C30 Hatchback 2012       0.67      0.76      0.71        41
                                   Volvo XC90 SUV 2007       0.97      0.65      0.78        43
                         smart fortwo Convertible 2012       0.64      0.72      0.68        40

                                              accuracy                           0.56      8041
                                             macro avg       0.66      0.56      0.57      8041
                                          weighted avg       0.66      0.56      0.57      8041

In [ ]:
# (!) Don't touch this cell

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")

Your model accuracy is 0.5637!